### Import Required Libraries and Set Up Environment Variables

In [114]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [115]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [116]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response

cme_response = requests.get(query_url)

In [117]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()



In [118]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [119]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.json_normalize(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID','startTime','linkedEvents']]

cme_df


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
...,...,...,...
5519,2024-05-01T06:36:00-CME-001,2024-05-01T06:36Z,None
5520,2024-05-01T11:36:00-CME-001,2024-05-01T11:36Z,None
5521,2024-05-01T12:36:00-CME-001,2024-05-01T12:36Z,None
5522,2024-05-01T17:36:00-CME-001,2024-05-01T17:36Z,None


In [120]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs

cme_df = cme_df[cme_df['linkedEvents'].notna()]
cme_df = cme_df.reset_index(drop=True)
cme_df


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
4,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...
...,...,...,...
1018,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,[{'activityID': '2024-04-26T00:17:00-IPS-001'}]
1019,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,[{'activityID': '2024-04-24T14:02:00-FLR-001'}]
1020,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,[{'activityID': '2024-04-25T17:03:00-FLR-001'}]
1021,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,[{'activityID': '2024-04-30T00:46:00-FLR-001'}]


In [121]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []


# Iterate over each index in the DataFrame
for i in cme_df.index:
    activityID = cme_df.loc[i, 'activityID']
    startTime = cme_df.loc[i, 'startTime']
    linkedEvents_col = cme_df.loc[i, 'linkedEvents']

    # Iterate over each dictionary in the list
    for item in linkedEvents_col:
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({'activityID':activityID, 'startTime':startTime, 'linkedEvents':item})
      
# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(expanded_rows)
expanded_cme_df


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'}
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'}
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'}
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'}


In [122]:
expanded_cme_df

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'}
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'}
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'}
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'}


In [123]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        activityID = input_dict.get('activityID', None)
        return activityID
    except(ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error processing input dictionary: {input_dict}. Error: {e}")


result = extract_activityID_from_dict(expanded_cme_df.loc[0, 'linkedEvents'])
print(result)

2013-05-04T04:52:00-IPS-001


In [124]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
expanded_cme_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [125]:
print(list(expanded_cme_df['GST_ActivityID'].unique()))

['2013-05-04T04:52:00-IPS-001', '2013-05-07T04:37:00-IPS-001', '2013-05-12T23:30:00-IPS-001', '2013-05-13T01:53:00-FLR-001', '2013-05-13T04:12:00-SEP-001', '2013-05-13T15:40:00-FLR-001', '2013-05-13T18:02:00-SEP-001', '2013-05-14T01:00:00-FLR-001', '2013-05-15T01:25:00-FLR-001', '2013-05-15T13:25:00-SEP-001', '2013-05-18T00:23:00-IPS-001', '2013-05-19T22:20:00-IPS-001', '2013-05-19T18:53:00-IPS-001', '2013-05-25T20:00:00-IPS-001', '2013-05-22T12:30:00-FLR-001', '2013-05-22T15:05:00-SEP-001', '2013-05-22T15:05:00-SEP-002', '2013-05-22T15:30:00-SEP-001', '2013-05-24T17:35:00-IPS-001', '2013-05-28T13:00:00-RBE-001', '2013-05-29T12:00:00-IPS-001', '2013-06-07T03:00:00-GST-001', '2013-06-07T22:32:00-FLR-001', '2013-06-19T15:08:00-IPS-001', '2013-06-20T11:13:00-IPS-001', '2013-06-21T06:09:00-SEP-001', '2013-06-21T17:37:00-IPS-001', '2013-06-23T20:10:00-SEP-001', '2013-06-27T16:17:00-IPS-001', '2013-07-05T01:00:00-IPS-001', '2013-07-05T07:04:00-IPS-001', '2013-07-11T01:05:00-IPS-001', '2013-0

In [126]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_cme_df = expanded_cme_df.dropna(subset=['GST_ActivityID'])
expanded_cme_df



,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [127]:
# print out the datatype of each column in this DataFrame:
print(expanded_cme_df.dtypes)


activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object


In [128]:
# Convert the 'GST_ActivityID' column to string format
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'])




In [129]:
# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df = expanded_cme_df.rename(
    columns={
        'startTime':'startTime_CME',
        'activityID':'cmeID'
    }
)

# Drop linkedEvents
expanded_cme_df = expanded_cme_df.drop(columns='linkedEvents')

# Verify that all steps were executed correctly
expanded_cme_df.info()
expanded_cme_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 40.3+ KB


,cmeID,startTime_CME,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01 03:12:00+00:00,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03 22:36:00+00:00,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09 19:29:00+00:00,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T04:12:00-SEP-001
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24 01:48:00+00:00,2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24 15:05:00+00:00,2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25 18:24:00+00:00,2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30 00:36:00+00:00,2024-04-30T00:46:00-FLR-001


In [130]:
print(list(expanded_cme_df['GST_ActivityID'].unique()))

['2013-05-04T04:52:00-IPS-001', '2013-05-07T04:37:00-IPS-001', '2013-05-12T23:30:00-IPS-001', '2013-05-13T01:53:00-FLR-001', '2013-05-13T04:12:00-SEP-001', '2013-05-13T15:40:00-FLR-001', '2013-05-13T18:02:00-SEP-001', '2013-05-14T01:00:00-FLR-001', '2013-05-15T01:25:00-FLR-001', '2013-05-15T13:25:00-SEP-001', '2013-05-18T00:23:00-IPS-001', '2013-05-19T22:20:00-IPS-001', '2013-05-19T18:53:00-IPS-001', '2013-05-25T20:00:00-IPS-001', '2013-05-22T12:30:00-FLR-001', '2013-05-22T15:05:00-SEP-001', '2013-05-22T15:05:00-SEP-002', '2013-05-22T15:30:00-SEP-001', '2013-05-24T17:35:00-IPS-001', '2013-05-28T13:00:00-RBE-001', '2013-05-29T12:00:00-IPS-001', '2013-06-07T03:00:00-GST-001', '2013-06-07T22:32:00-FLR-001', '2013-06-19T15:08:00-IPS-001', '2013-06-20T11:13:00-IPS-001', '2013-06-21T06:09:00-SEP-001', '2013-06-21T17:37:00-IPS-001', '2013-06-23T20:10:00-SEP-001', '2013-06-27T16:17:00-IPS-001', '2013-07-05T01:00:00-IPS-001', '2013-07-05T07:04:00-IPS-001', '2013-07-11T01:05:00-IPS-001', '2013-0

In [131]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library. 

cme_df_gst_filtered = expanded_cme_df[expanded_cme_df['GST_ActivityID'].str.contains('GST',na=False)]
cme_df_gst_filtered 



,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...
1519,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
1645,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
1665,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
1669,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


### GST Data

In [132]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_query_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [133]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_query_url)


In [134]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json, indent=4))


[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
                "activityID": "2013-06-02T20:24:

In [135]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.json_normalize(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID','startTime','linkedEvents']]


In [136]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset='linkedEvents').reset_index(drop=True)
gst_df


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
3,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
4,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...
...,...,...,...
100,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
101,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
102,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
103,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [137]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df_explode = gst_df.explode('linkedEvents', ignore_index=True).dropna()
gst_df_explode


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}
...,...,...,...
200,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'}
201,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T14:10:00-IPS-001'}
202,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T16:25:00-MPC-001'}
203,2024-04-19T18:00:00-GST-001,2024-04-19T18:00Z,{'activityID': '2024-04-15T06:48:00-CME-001'}


In [138]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df_explode['CME_ActivityID'] = gst_df_explode['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df_explode = gst_df_explode.dropna(subset='CME_ActivityID')
gst_df_explode


,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001
...,...,...,...,...
200,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'},2024-03-23T01:48:00-CME-001
201,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T14:10:00-IPS-001'},2024-03-24T14:10:00-IPS-001
202,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T16:25:00-MPC-001'},2024-03-24T16:25:00-MPC-001
203,2024-04-19T18:00:00-GST-001,2024-04-19T18:00Z,{'activityID': '2024-04-15T06:48:00-CME-001'},2024-04-15T06:48:00-CME-001


In [139]:
# Convert the 'CME_ActivityID' column to string format 
gst_df_explode['CME_ActivityID'] = gst_df_explode['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
gst_df_explode['gstID'] = gst_df_explode['gstID'].astype(str)

# Convert startTime to datetime format
gst_df_explode['startTime'] = pd.to_datetime(gst_df_explode['startTime'])

# Rename startTime to startTime_GST '
gst_df_explode = gst_df_explode.rename(columns={'startTime':'startTime_GST'})

# Drop linkedEvents
gst_df_explode = gst_df_explode.drop(columns='linkedEvents')

# Verify that all steps were executed correctly
gst_df_explode.info()
gst_df_explode


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.9+ KB


,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001
...,...,...,...
200,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001
201,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-24T14:10:00-IPS-001
202,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-24T16:25:00-MPC-001
203,2024-04-19T18:00:00-GST-001,2024-04-19 18:00:00+00:00,2024-04-15T06:48:00-CME-001


In [140]:
print(list(gst_df_explode['CME_ActivityID'].unique()))

['2013-05-31T15:45:00-HSS-001', '2013-06-02T20:24:00-CME-001', '2013-09-29T22:40:00-CME-001', '2013-10-02T01:54:00-IPS-001', '2013-10-02T02:47:00-MPC-001', '2013-12-04T23:12:00-CME-001', '2013-12-07T21:30:00-HSS-001', '2014-02-16T14:15:00-CME-001', '2014-02-19T03:10:00-IPS-001', '2014-02-18T01:25:00-CME-001', '2014-02-20T02:50:00-IPS-001', '2014-02-20T05:14:00-MPC-001', '2014-02-25T01:25:00-CME-001', '2014-02-27T16:10:00-IPS-001', '2014-06-04T13:25:00-CME-001', '2014-06-07T16:09:00-IPS-001', '2014-06-08T05:08:00-MPC-001', '2014-08-15T21:12:00-CME-001', '2014-08-19T05:57:00-IPS-001', '2014-09-10T18:18:00-CME-001', '2014-09-12T15:26:00-IPS-001', '2015-01-07T05:24:00-IPS-001', '2015-01-07T08:40:00-MPC-001', '2015-03-15T02:00:00-CME-001', '2015-03-17T04:05:00-IPS-001', '2015-03-16T00:00:00-HSS-001', '2015-03-21T20:25:00-IPS-001', '2015-05-12T17:20:00-HSS-001', '2015-06-08T00:16:00-HSS-001', '2015-06-21T02:48:00-CME-001', '2015-06-22T17:59:00-IPS-001', '2015-06-22T18:36:00-CME-001', '2015-0

In [141]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
cleaned_gst_df = gst_df_explode[gst_df_explode['CME_ActivityID'].str.contains('CME', na=False)]
cleaned_gst_df


,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
...,...,...,...
192,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001
195,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001
199,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001
200,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001


### Merge both datatsets

In [148]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = cleaned_gst_df.merge(
    cme_df_gst_filtered,
    left_on=['gstID','CME_ActivityID'],
    right_on= ['GST_ActivityID','cmeID'],
    how = "inner"
)
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


In [150]:
# Verify that the new DataFrame has the same number of rows as cme and gst
cme_df_gst_filtered.info()
cleaned_gst_df.info()
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 21 to 1689
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           61 non-null     object             
 1   startTime_CME   61 non-null     datetime64[ns, UTC]
 2   GST_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 1 to 203
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     object             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #

### Computing the time it takes for a CME to cause a GST

In [153]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2 days 04:20:00
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,3 days 00:48:00
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2 days 12:45:00
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2 days 01:35:00
...,...,...,...,...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001,3 days 12:22:00
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001,4 days 00:12:00
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001,1 days 10:35:00
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001,1 days 10:12:00


In [155]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_stats = merged_df['timeDiff'].describe()
print(time_stats)
mean = merged_df['timeDiff'].mean()
median = merged_df['timeDiff'].median()

print(f"Mean is {mean}")
print(f"Median is {median}")


count                           61
mean     2 days 23:29:26.557377049
std      0 days 23:53:09.336914240
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 19:51:00
75%                3 days 13:35:00
max                6 days 03:00:00
Name: timeDiff, dtype: object
Mean is 2 days 23:29:26.557377049
Median is 2 days 19:51:00


### Exporting data in csv format

In [157]:
# Export data to CSV without the index
merged_df.to_csv('output/NASA_output.csv',index=False)
